# pip & import

In [ ]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.3 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import category_encoders as ce

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
apply_train_df = pd.read_csv('/content/drive/MyDrive/Dacon(KMU)/data/apply_train.csv')
resume_train_df = pd.read_csv('/content/drive/MyDrive/Dacon(KMU)/data/resume.csv')
certificate_df = pd.read_csv('/content/drive/MyDrive/Dacon(KMU)/data/resume_certificate.csv')
education_train_df = pd.read_csv('/content/drive/MyDrive/Dacon(KMU)/data/resume_education.csv')
language_train_df = pd.read_csv('/content/drive/MyDrive/Dacon(KMU)/data/resume_language.csv')
company_train_df = pd.read_csv('/content/drive/MyDrive/Dacon(KMU)/data/company.csv')
recruitment_train_df = pd.read_csv('/content/drive/MyDrive/Dacon(KMU)/data/recruitment.csv')

In [ ]:
apply_matrix = apply_train_df.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)

In [ ]:
certificate_df

,resume_seq,certificate_contents
0,U06421,손해보험사
1,U01278,2종운전면허증
2,U01008,자동차운전면허 2종 보통
3,U03248,운전면허2종보통
4,U07483,메이크업3급
...,...,...
12970,U07453,컬러리스트산업기사
12971,U06358,컴퓨터그래픽스운용기능사
12972,U03248,수치제어밀링기능사
12973,U08157,주산2급


In [ ]:
# resume_certificate 정렬 (resume_seq 기준)
resume_certificate = certificate_df.sort_values(by='resume_seq')
# certificate_contents에 결측값이 존재하는 행 제거
resume_certificate = resume_certificate.dropna(subset=['certificate_contents'])
# 이력서번호(resume_seq)기준으로 groupby
# 이력서번호(resume_seq)의 해당하는 자격증들을 세미콜론(;)으로 join해서 적용(apply)
resume_certificate = resume_certificate.groupby('resume_seq')['certificate_contents'].apply(';'.join).reset_index()

In [ ]:
# merged_resume = resume, resume_certificate, resume_education, resume_language
merged_certificate = pd.merge(resume_train_df, resume_certificate, on='resume_seq', how='left')

In [ ]:
merged_certificate.columns

Index(['resume_seq', 'reg_date', 'updated_date', 'degree', 'graduate_date',
       'hope_salary', 'last_salary', 'text_keyword', 'job_code_seq1',
       'job_code_seq2', 'job_code_seq3', 'career_month', 'career_job_code',
       'certificate_contents'],
      dtype='object')

In [ ]:
merged_certificate.drop(merged_certificate.columns.difference(['resume_seq','certificate_contents']), 1, inplace=True)

In [ ]:
merged_certificate

,resume_seq,certificate_contents
0,U00606,NaN
1,U00509,NaN
2,U02012,중등 2급 정교사
3,U04599,MCAS MASTER 2007;VMD MASTER
4,U07573,워드프로세서 2급;컴퓨터그래픽스운용기능사
...,...,...
8477,U04890,NaN
8478,U05910,MOS(엑셀;워드;엑서스);샵마스터 3급;파워포인트
8479,U01468,자동차 운전면허증 1종보통
8480,U05315,1종보통 운전면허증


# 함수 정의 & 글로벌 변수

In [ ]:
def recall5(answer_df, submission_df):

    primary_col = answer_df.columns[0]
    secondary_col = answer_df.columns[1]

    # submission의 예측이 각각 5개인지 확인
    prediction_counts = submission_df.groupby(primary_col).size()
    if not all(prediction_counts == 5):
        raise ValueError(f"Each {primary_col} should have exactly 5 {secondary_col} predictions.")


    # submission의 예측된 값들에 null값이 있는지 확인
    if submission_df[secondary_col].isnull().any():
        raise ValueError(f"Predicted {secondary_col} contains NULL values.")

    # 예측값에 중복이 있는지 확인
    duplicated_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].duplicated().any())
    if duplicated_preds.any():
        raise ValueError(f"Predicted {secondary_col} contains duplicates for some {primary_col}.")


    # Filter the submission dataframe based on the primary_col present in the answer dataframe
    # primary_col 즉 resume_seq가 양측에 있는지 확인 후 남김
    submission_df = submission_df[submission_df[primary_col].isin(answer_df[primary_col])]

    # For each primary_col, get the top 5 predicted secondary_col values
    #
    top_5_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].head(5).tolist()).to_dict()

    # Convert the answer_df to a dictionary for easier lookup
    true_dict = answer_df.groupby(primary_col).apply(lambda x: x[secondary_col].tolist()).to_dict()


    individual_recalls = []
    for key, val in true_dict.items():
        if key in top_5_preds:
            correct_matches = len(set(true_dict[key]) & set(top_5_preds[key]))
            individual_recall = correct_matches / min(len(val), 5) # 공정한 평가를 가능하게 위하여 분모(k)를 'min(len(val), 5)' 로 설정함
            individual_recalls.append(individual_recall)


    recall = np.mean(individual_recalls)
    return recall

In [ ]:
def minimum_condition(resume, company):
  if(resume['degree'] < company['education']):
    return False
  else:
    return True
   # if(resume['career_month']>company['career_end'] or resume['career_month']<company['career_start']): 다 0임
     # return False
   # else:

In [ ]:
def check_salary(salary):
  if salary>3000:
    return 3
  elif salary>2000:
    return 2
  elif salary>1000:
    return 1
  else:
    return 0

In [ ]:
def check_career_month(career_month):
  if career_month>120:
    return 3
  elif career_month>60:
    return 2
  elif career_month>0:
    return 1
  else:
    return 0

In [ ]:
def check_univ_score(univ_score):
  if univ_score>=90:
    return 90
  elif univ_score>=80:
    return 80
  elif univ_score>=70:
    return 70
  elif univ_score>=60:
    return 60
  elif univ_score>=50:
    return 50
  elif univ_score>=40:
    return 40
  elif univ_score>=30:
    return 30
  elif univ_score>=20:
    return 20
  elif univ_score>=10:
    return 10
  else:
    return 0

#전공 이원화

In [ ]:
education_train_df['univ_major']

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
            ...     
8477    섬우패션학부의류패션전공
8478             NaN
8479             NaN
8480      의류학과생활과학계열
8481             NaN
Name: univ_major, Length: 8482, dtype: object

In [ ]:
education_train = education_train_df.copy()

In [ ]:
education_train_df['univ_major'] = education_train_df['univ_major'].fillna('X').astype(str)

In [ ]:
def classify_major(x):
    if '디자인' in x or '의류' in x or '섬유' in x or '예술' in x or '공예' in x:
        return 1
    elif '도예' in x or '의상' in x or '미술' in x or '아트' in x or 'art' in x:
        return 1
    elif 'CLOTH' in x or 'Design' in x or 'Fashion' in x or '동양화' in x or '한국화' in x:
        return 1
    elif '조형' in x or '공간' in x or '스타일' in x or '회화' in x:
        return 1
    else:
        return 0

# education_train['univ_major'] = education_train['univ_major'].astype(str)
education_train_df['univ_major'] = education_train_df['univ_major'].apply(classify_major)

In [ ]:
education_train_df

,resume_seq,hischool_type_seq,hischool_special_type,hischool_nation,hischool_gender,hischool_location_seq,univ_type_seq1,univ_type_seq2,univ_transfer,univ_location,univ_major,univ_sub_major,univ_major_type,univ_score
0,U01419,21,일반고,사립,남자학교,3,5,5,0,3,0,NaN,9,60.0
1,U03375,21,일반고,사립,여자학교,3,5,5,0,3,0,NaN,4,80.0
2,U06523,21,일반고,사립,남여공학,3,5,5,0,3,0,NaN,8,70.0
3,U06619,21,일반고,사립,남여공학,5,5,5,0,5,0,NaN,8,80.0
4,U05015,16,특성화고,공립,남여공학,3,5,5,0,3,0,NaN,9,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,U06326,21,일반고,사립,여자학교,9,5,5,0,7,1,NaN,10,70.0
8478,U07284,0,기타,기타,기타,0,5,5,0,11,0,NaN,3,70.0
8479,U04109,21,일반고,공립,남여공학,5,0,0,0,0,0,NaN,9,80.0
8480,U01863,21,일반고,사립,여자학교,15,3,5,0,15,1,NaN,9,80.0


# merge file

In [ ]:
certificate_education_merged_df = pd.merge(merged_certificate, education_train_df, how='left', left_on='resume_seq', right_on='resume_seq')

In [ ]:
resume_education_merged_df = pd.merge(resume_train_df, certificate_education_merged_df, how='left', left_on='resume_seq', right_on='resume_seq')

In [ ]:
merged_resume_df = pd.merge(apply_train_df, resume_education_merged_df, how='left', left_on='resume_seq', right_on='resume_seq')

merged_df = pd.merge(merged_resume_df, recruitment_train_df, how='left', left_on='recruitment_seq', right_on='recruitment_seq')

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57946 entries, 0 to 57945
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   resume_seq             57946 non-null  object 
 1   recruitment_seq        57946 non-null  object 
 2   reg_date               57946 non-null  object 
 3   updated_date           57946 non-null  object 
 4   degree                 57946 non-null  int64  
 5   graduate_date          57946 non-null  int64  
 6   hope_salary            57946 non-null  float64
 7   last_salary            57946 non-null  float64
 8   text_keyword_x         56744 non-null  object 
 9   job_code_seq1          57946 non-null  object 
 10  job_code_seq2          2937 non-null   object 
 11  job_code_seq3          1126 non-null   object 
 12  career_month           57946 non-null  int64  
 13  career_job_code        54557 non-null  object 
 14  certificate_contents   42235 non-null  object 
 15  hi

In [ ]:
merged_df.columns

Index(['resume_seq', 'recruitment_seq', 'reg_date', 'updated_date', 'degree',
       'graduate_date', 'hope_salary', 'last_salary', 'text_keyword_x',
       'job_code_seq1', 'job_code_seq2', 'job_code_seq3', 'career_month',
       'career_job_code', 'certificate_contents', 'hischool_type_seq',
       'hischool_special_type', 'hischool_nation', 'hischool_gender',
       'hischool_location_seq', 'univ_type_seq1', 'univ_type_seq2',
       'univ_transfer', 'univ_location', 'univ_major', 'univ_sub_major',
       'univ_major_type', 'univ_score', 'address_seq1', 'address_seq2',
       'address_seq3', 'career_end', 'career_start', 'check_box_keyword',
       'education', 'major_task', 'qualifications', 'text_keyword_y'],
      dtype='object')

# filter data & merge

##  자격증 개수 정리

In [ ]:
# 자격증 취득 개수 세기
def count_certificates(certificates):
    if pd.isnull(certificates):
        return 0
    elif ';' not in certificates:
        return 1
    else:
        return certificates.count(';') + 1

In [ ]:
# 'count_certificate' 열 추가
merged_df['count_certificate'] = merged_df['certificate_contents'].apply(count_certificates)

In [ ]:
merged_df['count_certificate'].unique()

array([ 2,  1,  0,  8,  5,  4,  3,  6,  7, 10,  9])

In [ ]:
merged_df['count_certificate'].value_counts()

1     18945
0     15711
2     10167
3      6329
4      3633
5      1726
6       901
7       301
8       151
9        56
10       26
Name: count_certificate, dtype: int64

In [ ]:
merged_df = merged_df.drop(['certificate_contents'], axis=1)

## 0. 고등학교 계열 정리

In [ ]:
study = merged_df[['resume_seq', 'hischool_special_type']]
merged_df['make_highschool_type'] = study['hischool_special_type'].replace({'일반고': '학업', '자율고': '학업', '특목고': '학업', '기타' : '직업', '특성화고': '직업'})

In [ ]:
merged_df['make_highschool_type']

0        학업
1        학업
2        학업
3        학업
4        학업
         ..
57941    학업
57942    직업
57943    학업
57944    직업
57945    학업
Name: make_highschool_type, Length: 57946, dtype: object

In [ ]:
merged_df['make_highschool_type'] = merged_df['make_highschool_type'].astype("category")

## 1. 요구학위에 맞지 않는 최종경력 이력서 filter

- 자격증 개수를 봐보자

In [ ]:
# 요구학위에 맞는 이력서 df + 요구학위가 맞지 않지만 고등학교 계열이 직업인 df + 자격증 3개 이상
degree_O_hi = merged_df[(merged_df['degree']>=merged_df['education'])|(merged_df['make_highschool_type']=='직업')|(merged_df['count_certificate']>=3)]

In [ ]:
degree_O_hi['resume_seq'].nunique()

8449

In [ ]:
# 요구학위와 직업학교 아닌 이력서 + 자격증 3개 이하인 df
degree_X_hi = merged_df[(merged_df['degree']<merged_df['education'])&(merged_df['make_highschool_type']!='직업')&(merged_df['count_certificate']<3)]
degree_X_hi['resume_seq'].nunique()

297

In [ ]:
# 조건에 맞지 않게 넣기만 한 사람들 filter
degree_hi_filter = degree_X_hi[~degree_X_hi['resume_seq'].isin(degree_O_hi['resume_seq'])]

In [ ]:
# 조건에 맞지 않게 넣기만 한 사람들
degree_hi_filter['resume_seq'].unique()

array(['U01879', 'U04770', 'U08118', 'U06182', 'U00670', 'U00996',
       'U04590', 'U06266', 'U04256', 'U01493', 'U03439', 'U03268',
       'U07207', 'U05988', 'U05362', 'U05368', 'U00535', 'U03895',
       'U00340', 'U01789', 'U05199', 'U05855', 'U01910', 'U02268',
       'U01358', 'U00151', 'U04660', 'U06894', 'U04406', 'U05745',
       'U00271', 'U02766', 'U05977'], dtype=object)

In [ ]:
# 조건에 맞지 않게 넣기만 한 사람들의 지원한 내용 df
degree_X = merged_df[merged_df['resume_seq'].isin(['U01879', 'U04770', 'U08118', 'U06182', 'U00670', 'U00996',
       'U04590', 'U06266', 'U04256', 'U01493', 'U03439', 'U03268',
       'U07207', 'U05988', 'U05362', 'U05368', 'U00535', 'U03895',
       'U00340', 'U01789', 'U05199', 'U05855', 'U01910', 'U02268',
       'U01358', 'U00151', 'U04660', 'U06894', 'U04406', 'U05745',
       'U00271', 'U02766', 'U05977'])]

In [ ]:
# 조건에 맞지 않게 넣기만 한 사람들의 자
degree_X['resume_seq'].nunique()

33

In [ ]:
# degree_O = 요구학위를 하나 이상이라도 맞게 넣은 사람들의 지원한 내용 df
# degree_X = 요구학위 맞지 않게 넣기만 한 사람들의 지원한 내용 df
# 이 둘은 열은 같은 컬럼이지만, 겹치는 내용이 없다. => concat 처리
merged_de_df = pd.concat([degree_O_hi, degree_X], ignore_index=True)

In [ ]:
# 이력서 번호 총 8482 확인하기
merged_de_df['resume_seq'].nunique()

8482

##ㅣ근무지 주소코드 - 고등학교 주소코드ㅣ<=2로 filter

In [ ]:
# 주소 거리가 5 이내로 지원한 내용 df
address_close = merged_de_df[((merged_de_df['address_seq1']-merged_de_df['hischool_location_seq']).abs())<=2]

In [ ]:
# 주소 거리가 5 넘는 지원한 내용 df
address_far = merged_de_df[((merged_de_df['address_seq1']-merged_de_df['hischool_location_seq']).abs())>2]

In [ ]:
# distance_5 컬럼 추가
address_close['distance_5'] = 0

In [ ]:
# distance_5 컬럼 추가
address_far['distance_5'] = 1

In [ ]:
resume_ad_merged_df = pd.concat([address_close, address_far], ignore_index=True)

In [ ]:
# 잘 합쳐졌는지 확인
resume_ad_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56849 entries, 0 to 56848
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   resume_seq             56849 non-null  object  
 1   recruitment_seq        56849 non-null  object  
 2   reg_date               56849 non-null  object  
 3   updated_date           56849 non-null  object  
 4   degree                 56849 non-null  int64   
 5   graduate_date          56849 non-null  int64   
 6   hope_salary            56849 non-null  float64 
 7   last_salary            56849 non-null  float64 
 8   text_keyword_x         55668 non-null  object  
 9   job_code_seq1          56849 non-null  object  
 10  job_code_seq2          2889 non-null   object  
 11  job_code_seq3          1114 non-null   object  
 12  career_month           56849 non-null  int64   
 13  career_job_code        53467 non-null  object  
 14  hischool_type_seq      56849 non-null 

# apply_matrix

In [ ]:
#학습, 검증 분리
train, test = [], []
apply_train_groupby = resume_ad_merged_df.groupby('resume_seq')['recruitment_seq'].apply(list)
for uid, iids in zip(apply_train_groupby.index.tolist(), apply_train_groupby.values.tolist()):
    if (len(iids)>1):
      for iid in iids[:-1]:
          train.append([uid,iid])
      test.append([uid, iids[-1]])
    else:
          train.append([uid,iid])
          test.append([uid, iid])

In [ ]:
apply_train = pd.DataFrame(train, columns=['resume_seq', 'recruitment_seq'])
apply_test = pd.DataFrame(test, columns=['resume_seq', 'recruitment_seq'])
apply_pred = merged_de_df.copy()

In [ ]:
# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
apply_train_item_matrix = apply_train.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
apply_train_similarity = cosine_similarity(apply_train_item_matrix) # 사용자 간의 유사성 계산

apply_test_item_matrix = apply_test.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
apply_test_similarity = cosine_similarity(apply_test_item_matrix) # 사용자 간의 유사성 계산

apply_pred_item_matrix = apply_pred.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
apply_pred_similarity = cosine_similarity(apply_pred_item_matrix) # 사용자 간의 유사성 계산

In [ ]:
apply_train_matrix = pd.DataFrame(apply_train_similarity) # DataFrame 생성
apply_test_matrix = pd.DataFrame(apply_test_similarity) # DataFrame 생성
apply_pred_matrix = pd.DataFrame(apply_pred_similarity) # DataFrame 생성

# 행과 열 이름 설정
apply_train_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
apply_train_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

apply_test_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
apply_test_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

apply_pred_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
apply_pred_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

In [ ]:
apply_train_matrix.shape

(8482, 8482)

# 회사 최소 조건

In [ ]:
resume_c = resume_train_df.copy()
recruitment_c = recruitment_train_df.copy()

In [ ]:
# 'resume_seq' 열을 기준으로 정렬
resume_c = resume_c.sort_values(by='resume_seq')
# 인덱스를 순차적으로 변경
resume_c['resume_seq'] = [f'U{i:05}' for i in range(1, len(resume_c) + 1)]
resume_c = resume_c.set_index('resume_seq')

# 'resume_seq' 열을 기준으로 정렬
recruitment_c = recruitment_c.sort_values(by='recruitment_seq')
# 인덱스를 순차적으로 변경
recruitment_c['recruitment_seq'] = [f'R{i:05}' for i in range(1, len(recruitment_c) + 1)]
recruitment_c = recruitment_c.set_index('recruitment_seq')

In [ ]:
resume_c = resume_c[['degree']]
recruitment_c = recruitment_c[['education']]

In [ ]:
apply_item_matrix_new = []
count = 0
tem_6 = [] # U05886이 6학년
tem_5 = [] # U08422이 5학년
tem_4 = [] # U04600이 4학년
tem_3 = [] # U03400이 3학년
tem_2 = [] # U04220이 2학년

for recruitment_seq in range(len(recruitment_c)): #
  if resume_c.loc['U05886']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        yy = False
  else:
        yy = True
  tem_6.append(yy)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U08422']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        yy = False
  else:
        yy = True
  tem_5.append(yy)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U04600']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        yy = False
  else:
        yy = True
  tem_4.append(yy)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U03400']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        yy = False
  else:
        yy = True
  tem_3.append(yy)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U04220']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        yy = False
  else:
        yy = True
  tem_2.append(yy)

# apply_item_matrix의 인덱스(=resume_seq)와 컬럼(=recruitment_seq)을 뽑아냄
for resume_seq in range(len(resume_c)):
    if resume_c.iloc[resume_seq]['degree'] == 6:
      apply_item_matrix_new.append(tem_6)
    elif resume_c.iloc[resume_seq]['degree'] == 5:
      apply_item_matrix_new.append(tem_5)
    elif resume_c.iloc[resume_seq]['degree'] == 4:
      apply_item_matrix_new.append(tem_4)
    elif resume_c.iloc[resume_seq]['degree'] == 3:
      apply_item_matrix_new.append(tem_3)
    else:
      apply_item_matrix_new.append(tem_2)

# 새로운 DataFrame 생성
apply_item_matrix_new = pd.DataFrame(apply_item_matrix_new, index=apply_matrix.index, columns=apply_matrix.columns)
apply_item_matrix_new

recruitment_seq,R00001,R00002,R00003,R00004,R00005,R00006,R00007,R00008,R00009,R00010,...,R06686,R06687,R06688,R06689,R06690,R06691,R06692,R06693,R06694,R06695
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00001,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00002,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00003,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00004,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00005,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08478,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U08479,True,True,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,False,False,True
U08480,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


# train 결과 도출

In [ ]:
# train_사용자 간의 유사성 계산
user_train_similarity = apply_train_similarity
user_predicted_scores = user_train_similarity.dot(apply_train_item_matrix) / np.array([np.abs(user_train_similarity).sum(axis=1)]).T

In [ ]:
item_train_similarity = cosine_similarity(apply_train_item_matrix.T)
item_train_predicted_scores = apply_train_item_matrix.dot(item_train_similarity)

In [ ]:
user_matrix = pd.DataFrame(user_train_similarity) # DataFrame 생성
# 행과 열 이름 설정
user_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
user_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

In [ ]:
sort_user_list = []
applied_jobs = []
alpha = 0.09

for idx, user in enumerate(user_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_job = apply_train[apply_train['resume_seq'] == user]['recruitment_seq'].values
    applied_jobs.append([user, applied_job])

    # 해당 사용자의 최종 유사도 (높은 유사도부터 정렬)
    sorted_user_indices = (user_predicted_scores[idx] + item_train_predicted_scores.loc[user]*alpha).argsort()[::-1]
    sort_user_list.append([user,sorted_user_indices])

In [ ]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []


for idx, user in enumerate(apply_train_item_matrix.index):
  re_applied_jobs = []
  # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
  sorted_job_indices = (user_predicted_scores[idx] + item_train_predicted_scores.loc[user]*alpha).argsort()[::-1]
  for idxx, j in enumerate(sorted_job_indices):
    if(len(re_applied_jobs) >= 5):
      break
    else:
      job = apply_train_item_matrix.columns[j]
      if (job not in applied_jobs[idx][1] and apply_item_matrix_new.loc[user].loc[job] and (job not in re_applied_jobs)):
        re_applied_jobs.append(job)
        recommendations.append([user, job])

In [ ]:
val_prediction = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

In [ ]:
recall5(apply_test,val_prediction)

0.12343786842725772

# pred 결과 도출

In [ ]:
# train_사용자 간의 유사성 계산
pred_user_similarity = apply_pred_similarity
pred_user_predicted_scores = pred_user_similarity.dot(apply_pred_item_matrix) / np.array([np.abs(pred_user_similarity).sum(axis=1)]).T

In [ ]:
pred_item_train_similarity = cosine_similarity(apply_pred_item_matrix.T)
pred_item_train_predicted_scores = apply_pred_item_matrix.dot(pred_item_train_similarity)

In [ ]:
pred_user_similarity.shape

In [ ]:
user_matrix = pd.DataFrame(pred_user_similarity) # DataFrame 생성
# 행과 열 이름 설정
user_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
user_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

In [ ]:
sort_user_list = []
applied_jobs = []
alpha = 0.09

for idx, user in enumerate(user_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_job = apply_pred[apply_pred['resume_seq'] == user]['recruitment_seq'].values
    applied_jobs.append([user, applied_job])

    # 해당 사용자의 최종 유사도 (높은 유사도부터 정렬)
    sorted_user_indices = (pred_user_predicted_scores[idx] + pred_item_train_predicted_scores.loc[user]*alpha).argsort()[::-1]
    sort_user_list.append([user,sorted_user_indices])

In [ ]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []


for idx, user in enumerate(apply_pred_item_matrix.index):
  re_applied_jobs = []
  # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
  sorted_job_indices = (pred_user_predicted_scores[idx] + pred_item_train_predicted_scores.loc[user]*alpha).argsort()[::-1]
  for idxx, j in enumerate(sorted_job_indices):
    if(len(re_applied_jobs) >= 5):
      break
    else:
      job = apply_pred_item_matrix.columns[j]
      if (job not in applied_jobs[idx][1] and apply_item_matrix_new.loc[user].loc[job] and (job not in re_applied_jobs)):
        re_applied_jobs.append(job)
        recommendations.append([user, job])

In [ ]:
# sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

In [ ]:
top_recommendations.to_csv('세 번째 test.csv', index=False)